**Table of contents**<a id='toc0_'></a>    
- [XGBoost 全面解析](#toc1_)    
  - [什么是XGBoost？](#toc1_1_)    
  - [算法原理](#toc1_2_)    
    - [Boosting 机制](#toc1_2_1_)    
    - [核心创新](#toc1_2_2_)    
    - [XGBoost与传统GBDT的区别](#toc1_2_3_)    
    - [目标函数](#toc1_2_4_)    
  - [目录](#toc1_3_)    
  - [什么是XGBoost](#toc1_4_)    
    - [XGBoost树的定义](#toc1_4_1_)    
    - [正则项：树的复杂度](#toc1_4_2_)    
    - [树该怎么长](#toc1_4_3_)    
    - [如何停止树的循环生成](#toc1_4_4_)    
  - [XGBoost与GBDT有什么不同](#toc1_5_)    
  - [为什么XGBoost要用泰勒展开，优势在哪里？](#toc1_6_)    
  - [代码实现](#toc1_7_)    
  - [参考文献](#toc1_8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [2]:
# First XGBoost model for Pima Indians dataset
from numpy import loadtxt


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import *

# print("Python:", sys.version)
import xgboost
from xgboost import XGBClassifier
import sklearn

import sys
import sklearn
import xgboost
import numpy as np

In [3]:

# 1. 加载数据
# 使用numpy的loadtxt函数加载CSV文件，delimiter=","表示用逗号分隔数据
dataset = loadtxt("pima-indians-diabetes.csv", delimiter=",")

# 2. 分割数据为特征(X)和标签(Y)
# 前8列是特征(输入变量)
X = dataset[:, 0:8]
# 第9列是标签(输出变量)
Y = dataset[:, 8]

# 3. 划分训练集和测试集
seed = 7  # 随机种子，保证每次划分结果一致
test_size = 0.33  # 测试集占比33%
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=test_size, random_state=seed  # 测试集比例  # 随机状态
)

# 4. 创建并训练XGBoost模型
# 初始化XGBoost分类器
model = XGBClassifier(
    # 以下是XGBoost 3.0+版本推荐的参数设置
    eval_metric="logloss",  # 指定评估指标为对数损失(适用于二分类)
    use_label_encoder=False,  # 禁用旧版标签编码器
    verbosity=1,  # 显示警告信息
)

# 训练模型
model.fit(
    X_train,  # 训练特征
    y_train,  # 训练标签
    # eval_set=[(X_test, y_test)],  # 验证集(用于早停和监控训练过程)
    # verbose=True  # 是否显示训练过程(调试时可开启)
)

# 5. 在测试集上进行预测
# 使用训练好的模型对测试集进行预测
y_pred = model.predict(X_test)
# 将预测概率四舍五入为0或1(因为这是二分类问题)
predictions = [round(value) for value in y_pred]

# 6. 评估模型性能
# 计算并打印准确率
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# 可选: 打印特征重要性
print("\n特征重要性:")
for i, score in enumerate(model.feature_importances_):
    print(f"特征 {i}: {score:.3f}")

Accuracy: 72.83%

特征重要性:
特征 0: 0.085
特征 1: 0.244
特征 2: 0.074
特征 3: 0.112
特征 4: 0.108
特征 5: 0.147
特征 6: 0.095
特征 7: 0.134


# <a id='toc1_'></a>[XGBoost 全面解析](#toc0_)

## <a id='toc1_1_'></a>[什么是XGBoost？](#toc0_)
XGBoost 既是机器学习库的名称，也是该库核心算法的名称。它是一种基于梯度提升框架(Gradient Boosting)的优化算法，专为**效率**和**性能**设计，在各类机器学习竞赛和工业应用中表现卓越。

## <a id='toc1_2_'></a>[算法原理](#toc0_)

### <a id='toc1_2_1_'></a>[Boosting 机制](#toc0_)
XGBoost 属于集成学习中的Boosting方法，boost方法核心思想如下：
- **串行训练**：依次构建多个弱学习器（多为决策树），前一个的输出是后一个的输入。
- **错误修正**：后续模型专注于纠正前序模型的错误
- **加权组合**：最终预测是所有弱学习器的加权和

boosting能有效降低偏差，但方差可能会升高，因为boosting专注于错误样本。

### <a id='toc1_2_2_'></a>[核心创新](#toc0_)
XGBoost 在传统GBDT基础上做了多项改进：

$$\text{Obj} = \sum L(y_i, \hat{y}_i) + \sum \Omega(f_k) $$

GBDT（Gradient Boosting Decision Tree，梯度提升决策树）是一种基于决策树的集成学习算法，通过**梯度提升**的方式迭代构建多个弱学习器，其是基于损失函数的迭代优化，更容易过拟合。

而XGBoost则是基于目标函数的迭代优化，通过控制模型复杂度来避免过拟合。

### <a id='toc1_2_3_'></a>[XGBoost与传统GBDT的区别](#toc0_)

| 特性 | XGBoost | 传统GBDT |
|------|---------|----------|
| **正则化** | 包含L1/L2正则项 | 通常无正则化 |
| **损失函数** | 支持自定义，使用二阶泰勒展开 | 一般使用一阶梯度 |
| **缺失值处理** | 自动学习最优处理方式 | 需要手动处理 |
| **并行化** | 特征并行，数据并行 | 仅支持数据并行 |
| **剪枝策略** | 后剪枝+最大深度限制 | 预剪枝为主 |

通过使用二阶泰勒展开，可以加速收敛，自动计算最优步长，训练次数减少30%到50%。

$$L(y_i, \hat{y}_i^{(t-1)}+f_t(x_i)) \approx L(y_i, \hat{y}_i^{(t-1)}) + g_i f_t(x_i) + \frac{1}{2}h_i f_t^2(x_i)$$

### <a id='toc1_2_4_'></a>[目标函数](#toc0_)
XGBoost的目标函数是损失函数加上正则化项，GBDT的目标函数是损失函数：

$$\text{Obj}(\theta) = \underbrace{\sum_{i=1}^n L(y_i, \hat{y}_i)}_{\text{损失函数}} + \underbrace{\sum_{k=1}^K \Omega(f_k)}_{\text{正则化项}}$$

其中损失函数一般是平方损失$(y_i - ŷ_i)^2$或绝对损失$|y_i - ŷ_i|$，$y_i$是样本的真实标签，$\hat{y}_i$是模型预测的标签，$K$是树的个数，$\Omega(f_k)$是正则项,式子如下：

$$\Omega(f_k) = \gamma T + \frac{1}{2}\lambda||w||^2 + \alpha||w||_1$$

| 组成部分           | 作用机制                             | 效果                             |
|--------------------|--------------------------------------|----------------------------------|
| **γT（复杂度控制）** | 每增加一个叶子节点，惩罚γ值          | 抑制树过深，防止模型过于复杂       |
| **L2项（λ）**       | 对叶子权重w的平方惩罚                | 使权重分布平滑，避免极端值         |
| **L1项（α）**       | 对叶子权重的绝对值惩罚               | 产生稀疏解，自动特征选择           |

## <a id='toc1_3_'></a>[目录](#toc0_)
- [1. 什么是XGBoost](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.3%20XGBoost#1-什么是xgboost)
  - [1.1 XGBoost树的定义](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.3%20XGBoost#11-xgboost树的定义)
  - [1.2 正则项：树的复杂度](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.3%20XGBoost#12-正则项树的复杂度)
  - [1.3 树该怎么长](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.3%20XGBoost#13-树该怎么长)
  - [1.4 如何停止树的循环生成](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.3%20XGBoost#14-如何停止树的循环生成)
- [2. XGBoost与GBDT有什么不同](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.3%20XGBoost#2-xgboost与gbdt有什么不同)
- [3. 为什么XGBoost要用泰勒展开，优势在哪里？](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.3%20XGBoost#3-为什么xgboost要用泰勒展开优势在哪里)
- [4. 代码实现](https://github.com/NLP-LOVE/ML-NLP/blob/master/Machine%20Learning/3.3%20XGBoost/3.3%20XGBoost.ipynb)
- [5. 参考文献](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.3%20XGBoost#5-参考文献)

## <a id='toc1_4_'></a>[什么是XGBoost](#toc0_)

XGBoost是陈天奇等人开发的一个开源机器学习项目，高效地实现了GBDT算法并进行了算法和工程上的许多改进，被广泛应用在Kaggle竞赛及其他许多机器学习竞赛中并取得了不错的成绩。

说到XGBoost，不得不提GBDT(Gradient Boosting Decision Tree)。因为XGBoost本质上还是一个GBDT，但是力争把速度和效率发挥到极致，所以叫X (Extreme) GBoosted。包括前面说过，两者都是boosting方法。

关于GBDT，这里不再提，可以查看我前一篇的介绍，[点此跳转](https://github.com/NLP-LOVE/ML-NLP/blob/master/Machine%20Learning/3.2%20GBDT/3.2%20GBDT.md)。

### <a id='toc1_4_1_'></a>[XGBoost树的定义](#toc0_)

先来举个**例子**，我们要预测一家人对电子游戏的喜好程度，考虑到年轻和年老相比，年轻更可能喜欢电子游戏，以及男性和女性相比，男性更喜欢电子游戏，故先根据年龄大小区分小孩和大人，然后再通过性别区分开是男是女，逐一给各人在电子游戏喜好程度上打分，如下图所示。

![](https://julyedu-img.oss-cn-beijing.aliyuncs.com/quesbase64153438577232516800.png)

就这样，训练出了2棵树tree1和tree2，类似之前gbdt的原理，两棵树的结论累加起来便是最终的结论，所以小孩的预测分数就是两棵树中小孩所落到的结点的分数相加：2 + 0.9 = 2.9。爷爷的预测分数同理：-1 + （-0.9）= -1.9。具体如下图所示：

![](https://julyedu-img.oss-cn-beijing.aliyuncs.com/quesbase64153438578739198433.png)

恩，你可能要拍案而起了，惊呼，这不是跟上文介绍的GBDT乃异曲同工么？

事实上，如果不考虑工程实现、解决问题上的一些差异，XGBoost与GBDT比较大的不同就是目标函数的定义。XGBoost的目标函数如下图所示：

![](https://julyedu-img.oss-cn-beijing.aliyuncs.com/quesbase64153438580139159593.png)

其中：

- 红色箭头所指向的L 即为损失函数（比如平方损失函数：![](https://latex.codecogs.com/gif.latex?l(y_i,y^i)=(y_i-y^i)^2))
- 红色方框所框起来的是正则项（包括L1正则、L2正则）
- 红色圆圈所圈起来的为常数项
- 对于f(x)，XGBoost利用泰勒展开三项，做一个近似。**f(x)表示的是其中一颗回归树。**

看到这里可能有些读者会头晕了，这么多公式，**我在这里只做一个简要式的讲解，具体的算法细节和公式求解请查看这篇博文，讲得很仔细**：[通俗理解kaggle比赛大杀器xgboost](https://blog.csdn.net/v_JULY_v/article/details/81410574)

XGBoost的**核心算法思想**不难，基本就是：

1. 不断地添加树，不断地进行特征分裂来生长一棵树，每次添加一个树，其实是学习一个新函数**f(x)**，去拟合上次预测的残差。
2. 当我们训练完成得到k棵树，我们要预测一个样本的分数，其实就是根据这个样本的特征，在每棵树中会落到对应的一个叶子节点，每个叶子节点就对应一个分数
3. 最后只需要将每棵树对应的分数加起来就是该样本的预测值。

显然，我们的目标是要使得树群的预测值![](https://latex.codecogs.com/gif.latex?y_i^{'})尽量接近真实值![](https://latex.codecogs.com/gif.latex?y_i)，而且有尽量大的泛化能力。类似之前GBDT的套路，XGBoost也是需要将多棵树的得分累加得到最终的预测得分（每一次迭代，都在现有树的基础上，增加一棵树去拟合前面树的预测结果与真实值之间的残差）。

![](https://julyedu-img.oss-cn-beijing.aliyuncs.com/quesbase64153438657261833493.png)

那接下来，我们如何选择每一轮加入什么 f 呢？答案是非常直接的，选取一个 f 来使得我们的目标函数尽量最大地降低。这里 f 可以使用泰勒展开公式近似。

![](https://julyedu-img.oss-cn-beijing.aliyuncs.com/quesbase6415343865867530120.png)

实质是把样本分配到叶子结点会对应一个obj，优化过程就是obj优化。也就是分裂节点到叶子不同的组合，不同的组合对应不同obj，所有的优化围绕这个思想展开。到目前为止我们讨论了目标函数中的第一个部分：训练误差。接下来我们讨论目标函数的第二个部分：正则项，即如何定义树的复杂度。

### <a id='toc1_4_2_'></a>[正则项：树的复杂度](#toc0_)

XGBoost对树的复杂度包含了两个部分：

- 一个是树里面叶子节点的个数T
- 一个是树上叶子节点的得分w的L2模平方（对w进行L2正则化，相当于针对每个叶结点的得分增加L2平滑，目的是为了避免过拟合）

![](https://julyedu-img.oss-cn-beijing.aliyuncs.com/quesbase64153438674199471483.png)

我们再来看一下XGBoost的目标函数（损失函数揭示训练误差 + 正则化定义复杂度）：

![](https://latex.codecogs.com/gif.latex?L(\phi)=\sum_{i}l(y_i^{'}-y_i)+\sum_k\Omega(f_t))

正则化公式也就是目标函数的后半部分，对于上式而言，![](https://latex.codecogs.com/gif.latex?y_i^{'})是整个累加模型的输出，正则化项∑kΩ(ft)是则表示树的复杂度的函数，值越小复杂度越低，泛化能力越强。

### <a id='toc1_4_3_'></a>[树该怎么长](#toc0_)

很有意思的一个事是，我们从头到尾了解了xgboost如何优化、如何计算，但树到底长啥样，我们却一直没看到。很显然，一棵树的生成是由一个节点一分为二，然后不断分裂最终形成为整棵树。那么树怎么分裂的就成为了接下来我们要探讨的关键。对于一个叶子节点如何进行分裂，XGBoost作者在其原始论文中给出了一种分裂节点的方法：**枚举所有不同树结构的贪心法**

不断地枚举不同树的结构，然后利用打分函数来寻找出一个最优结构的树，接着加入到模型中，不断重复这样的操作。这个寻找的过程使用的就是**贪心算法**。选择一个feature分裂，计算loss function最小值，然后再选一个feature分裂，又得到一个loss function最小值，你枚举完，找一个效果最好的，把树给分裂，就得到了小树苗。

总而言之，XGBoost使用了和CART回归树一样的想法，利用贪婪算法，遍历所有特征的所有特征划分点，不同的是使用的目标函数不一样。具体做法就是分裂后的目标函数值比单子叶子节点的目标函数的增益，同时为了限制树生长过深，还加了个阈值，只有当增益大于该阈值才进行分裂。从而继续分裂，形成一棵树，再形成一棵树，**每次在上一次的预测基础上取最优进一步分裂/建树。**

### <a id='toc1_4_4_'></a>[如何停止树的循环生成](#toc0_)

凡是这种循环迭代的方式必定有停止条件，什么时候停止呢？简言之，设置树的最大深度、当样本权重和小于设定阈值时停止生长以防止过拟合。具体而言，则

1. 当引入的分裂带来的增益小于设定阀值的时候，我们可以忽略掉这个分裂，所以并不是每一次分裂loss function整体都会增加的，有点预剪枝的意思，阈值参数为（即正则项里叶子节点数T的系数）；
2. 当树达到最大深度时则停止建立决策树，设置一个超参数max_depth，避免树太深导致学习局部样本，从而过拟合；
3. 样本权重和小于设定阈值时则停止建树。什么意思呢，即涉及到一个超参数-最小的样本权重和min_child_weight，和GBM的 min_child_leaf 参数类似，但不完全一样。大意就是一个叶子节点样本太少了，也终止同样是防止过拟合；

## <a id='toc1_5_'></a>[XGBoost与GBDT有什么不同](#toc0_)

除了算法上与传统的GBDT有一些不同外，XGBoost还在工程实现上做了大量的优化。总的来说，两者之间的区别和联系可以总结成以下几个方面。 

1. GBDT是机器学习算法，XGBoost是该算法的工程实现。
2. 在使用CART作为基分类器时，XGBoost显式地加入了正则项来控制模 型的复杂度，有利于防止过拟合，从而提高模型的泛化能力。
3. GBDT在模型训练时只使用了代价函数的一阶导数信息，XGBoost对代 价函数进行二阶泰勒展开，可以同时使用一阶和二阶导数。
4. 传统的GBDT采用CART作为基分类器，XGBoost支持多种类型的基分类 器，比如线性分类器。
5. 传统的GBDT在每轮迭代时使用全部的数据，XGBoost则采用了与随机 森林相似的策略，支持对数据进行采样。
6. 传统的GBDT没有设计对缺失值进行处理，XGBoost能够自动学习出缺 失值的处理策略。

## <a id='toc1_6_'></a>[为什么XGBoost要用泰勒展开，优势在哪里？](#toc0_)

XGBoost使用了一阶和二阶偏导, 二阶导数有利于梯度下降的更快更准. 使用泰勒展开取得函数做自变量的二阶导数形式, 可以在不选定损失函数具体形式的情况下, 仅仅依靠输入数据的值就可以进行叶子分裂优化计算, 本质上也就把损失函数的选取和模型算法优化/参数选择分开了. 这种去耦合增加了XGBoost的适用性, 使得它按需选取损失函数, 可以用于分类, 也可以用于回归。

## <a id='toc1_7_'></a>[代码实现](#toc0_)

GitHub：[点击进入](https://github.com/NLP-LOVE/ML-NLP/blob/master/Machine%20Learning/3.3%20XGBoost/3.3%20XGBoost.ipynb)

## <a id='toc1_8_'></a>[参考文献](#toc0_)

[通俗理解kaggle比赛大杀器xgboost](https://blog.csdn.net/v_JULY_v/article/details/81410574)

> 作者：[@mantchs](https://github.com/NLP-LOVE/ML-NLP)
>
> GitHub：[https://github.com/NLP-LOVE/ML-NLP](https://github.com/NLP-LOVE/ML-NLP)
>
> 欢迎大家加入讨论！共同完善此项目！群号：【541954936】<a target="_blank" href="//shang.qq.com/wpa/qunwpa?idkey=863f915b9178560bd32ca07cd090a7d9e6f5f90fcff5667489697b1621cecdb3"><img border="0" src="http://pub.idqqimg.com/wpa/images/group.png" alt="NLP面试学习群" title="NLP面试学习群"></a>
